In [6]:
import numpy as np

In [7]:
def read_BankChurners(path):
    dtypes = [
    ('CLIENTNUM', int),
    ('Attrition_Flag', 'U20'),  
    ('Customer_Age', int),
    ('Gender', 'U10'),
    ('Dependent_count', int),
    ('Education_Level', 'U30'),
    ('Marital_Status', 'U20'),
    ('Income_Category', 'U20'),
    ('Card_Category', 'U20'),
    ('Months_on_book', int),
    ('Total_Relationship_Count', int),
    ('Months_Inactive_12_mon', int),
    ('Contacts_Count_12_mon', int),
    ('Credit_Limit', float),
    ('Total_Revolving_Bal', int),
    ('Avg_Open_To_Buy', float),
    ('Total_Amt_Chng_Q4_Q1', float),
    ('Total_Trans_Amt', int),
    ('Total_Trans_Ct', int),
    ('Total_Ct_Chng_Q4_Q1', float),
    ('Avg_Utilization_Ratio', float)
]
    data = np.genfromtxt(
    path, 
    delimiter=',', 
    dtype=dtypes, 
    skip_header=1,   
    encoding='utf-8'
)
    col_names = [dtype[0] for dtype in dtypes]
    return data, col_names

path = '../data/raw/BankChurners.csv'
data, col_names = read_BankChurners(path)

In [8]:
numeric_cols = [name for name in data.dtype.names if data.dtype[name].kind in ('i', 'f')]
str_cols = [name for name in data.dtype.names if data.dtype[name].kind in ('U')]

for col in str_cols:
    data[col] = np.char.strip(data[col], '"')

In [9]:
data

array([(768805383, 'Existing Customer', 45, 'M', 3, 'High School', 'Married', '$60K - $80K', 'Blue', 39, 5, 1, 3, 12691.,  777, 11914., 1.335,  1144, 42, 1.625, 0.061),
       (818770008, 'Existing Customer', 49, 'F', 5, 'Graduate', 'Single', 'Less than $40K', 'Blue', 44, 6, 1, 2,  8256.,  864,  7392., 1.541,  1291, 33, 3.714, 0.105),
       (713982108, 'Existing Customer', 51, 'M', 3, 'Graduate', 'Married', '$80K - $120K', 'Blue', 36, 4, 1, 0,  3418.,    0,  3418., 2.594,  1887, 20, 2.333, 0.   ),
       ...,
       (716506083, 'Attrited Customer', 44, 'F', 1, 'High School', 'Married', 'Less than $40K', 'Blue', 36, 5, 3, 4,  5409.,    0,  5409., 0.819, 10291, 60, 0.818, 0.   ),
       (717406983, 'Attrited Customer', 30, 'M', 2, 'Graduate', 'Unknown', '$40K - $60K', 'Blue', 36, 4, 3, 3,  5281.,    0,  5281., 0.535,  8395, 62, 0.722, 0.   ),
       (714337233, 'Attrited Customer', 43, 'F', 2, 'Graduate', 'Married', 'Less than $40K', 'Silver', 25, 6, 2, 4, 10388., 1961,  8427., 0.703, 1

In [ ]:
def remove_outlier(col):
    Q1, Q3 = np.percentile(col, [25, 75])
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    for i in col:
        if i < lower:
            i = lower
        elif i > upper:
            i = upper
    return col

def minmax_scaler(col):
    min_val = col.min()
    max_val = col.max()
    if max_val != min_val:
        return (col - min_val) / (max_val - min_val)
    else:
        return np.zeros_like(col)
    
def feature_encode(col):
    unique_vals = np.unique(col)
    mapping = {val: idx for idx, val in enumerate(unique_vals)}
    encoded = np.array([mapping[val] for val in col])
    return encoded, mapping

def encode_label(y):
    return (y == "Attrited Customer").astype(int)

In [12]:
for col in numeric_cols:
    print( remove_outlier(data[col]))
    data[col] = remove_outlier(data[col])
    data[col] = minmax_scaler(data[col])

mappings = {}
categorical_cols = ["Education_Level, Marital_Status, Income_Category, Card_Category"]
for col in categorical_cols:
    arr = data[col].astype(str)
    encoded, mapping = feature_encode(arr)
    data[col] = encoded
    mappings[col] = mapping

data["Attrition_Flag"] = encode_label(data["Attrition_Flag"])

(622876626.75, 863303676.75)


ValueError: could not broadcast input array from shape (2,) into shape (10127,)